## Azure OpenAI Code Interpreter Research

### 共通部

In [1]:
import json
from openai import AzureOpenAI
import time
from IPython.display import clear_output
import yaml

In [59]:
# あるスレッドに対してリクエストを送信した際のレスポンスを取得
def get_response(client: AzureOpenAI, assistant, thread, request):
    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=request
    )

    run = client.beta.threads.runs.create(
        thread_id=thread.id,
        assistant_id=assistant.id
    )

    start_time = time.time()

    status = run.status

    while status not in ["completed", "cancelled", "expired", "failed"]:
        time.sleep(5)
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id
        )
        print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
        status = run.status
        print(f'Status: {status}')
        clear_output(wait=True)

    messages = client.beta.threads.messages.list(
        thread_id=thread.id
    )

    print(f'Status: {status}')
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    print(messages.model_dump_json(indent=2))

In [2]:
file_path = "credentials.yaml"

with open(file_path, "r") as file:
    credentials = yaml.load(file, Loader=yaml.FullLoader)

In [60]:
client = AzureOpenAI(
    api_key=credentials["openai_gpt4"]["key"],
    api_version=credentials["openai_gpt4"]["version"],
    azure_endpoint=credentials["openai_gpt4"]["endpoint"]
)

### Code Interpreter が得意とする標準的なタスクを解く

#### タスク1： 連立方程式を解く

In [3]:
request = "レンガ1個の重さはレンガ1個の30%にGPU5個を加えた重さだ。1トンには100個のレンガと200個のGPUがあるが、レンガ1個の重さは？"

In [ ]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model"
)

In [6]:
thread = client.beta.threads.create()

Thread(id='thread_jB8E2MnqO4cDEUbejJVTq4nq', created_at=1708404653, metadata={}, object='thread')


In [11]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 0 minutes 47 seconds
{
  "data": [
    {
      "id": "msg_9g6KcPBhNRSGXMZCY9WVIqUc",
      "assistant_id": "asst_ScnkLIbPQTyQAZZCwjimhg7n",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "レンガ1個の重さは約7.8125キログラムです。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708405089,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_FqqCBDxBkHAhGLiu8G3Nbo31",
      "thread_id": "thread_jB8E2MnqO4cDEUbejJVTq4nq"
    },
    {
      "id": "msg_lQCw8D3nMYaUpWigjedwxwKC",
      "assistant_id": "asst_ScnkLIbPQTyQAZZCwjimhg7n",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "レンガ1個の重さを \\( R \\) キログラムとし、GPU1個の重さを \\( G \\) キログラムとしましょう。問題の文章によれば、次の等式が成り立ちます：\n\n\\[\nR = 0.3R + 5G\n\\]\n\nこれは、レンガ1個の重さが、その30%にGPU5個の重さを加えたものに等しいという意味です。しかし、これだけでは2つの未知数に対して1つの式しかないため、解

In [ ]:
get_response(client=client, assistant=assistant, thread=thread, request="使用したコードを教えてください")

In [16]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

{
  "data": [
    {
      "id": "msg_w3BXCqBvnvngxLQN5azq2qTy",
      "assistant_id": "asst_ScnkLIbPQTyQAZZCwjimhg7n",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下はレンガ1個の重さを解くために使用したPythonコードと、それに使用された `sympy` モジュールのコードです：\n\n```python\nfrom sympy import symbols, Eq, solve\n\n# 未知数を定義する\nR, G = symbols('R G')\n\n# 与えられた方程式を定義する\neq1 = Eq(R, 0.3*R + 5*G)  # レンガ1個の重さはその30% + GPU5個の重さ\neq2 = Eq(100*R + 200*G, 1000)  # 1トンには100個のレンガと200個のGPUがある\n\n# 方程式を解く\nsolutions = solve((eq1, eq2), (R, G))\n\n# レンガ1個の重さを取得する\nbrick_weight = solutions[R]\nbrick_weight.evalf()\n```\n\nコードではまず `sympy` モジュールから `symbols`, `Eq`, `solve` 関数をインポートします。`symbols` 関数を使って二つの未知数 \\( R \\) と \\( G \\) を定義し、`Eq` 関数を使って二つの方程式 `eq1` と `eq2` を設定します。そして `solve` 関数を使ってこれらの方程式を同時に解き、`R` (レンガ1個の重さ) を求めます。`evalf()` メソッドは数値解を求めるために使われます。\n\n実際に実行されたコードにより、レンガ1個の重さは約7.8125キログラムであると求めることができました。"
          },
          "type": "text"
        }
      ],
      "cre

In [17]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

もちろんです。以下はレンガ1個の重さを解くために使用したPythonコードと、それに使用された `sympy` モジュールのコードです：

```python
from sympy import symbols, Eq, solve

# 未知数を定義する
R, G = symbols('R G')

# 与えられた方程式を定義する
eq1 = Eq(R, 0.3*R + 5*G)  # レンガ1個の重さはその30% + GPU5個の重さ
eq2 = Eq(100*R + 200*G, 1000)  # 1トンには100個のレンガと200個のGPUがある

# 方程式を解く
solutions = solve((eq1, eq2), (R, G))

# レンガ1個の重さを取得する
brick_weight = solutions[R]
brick_weight.evalf()
```

コードではまず `sympy` モジュールから `symbols`, `Eq`, `solve` 関数をインポートします。`symbols` 関数を使って二つの未知数 \( R \) と \( G \) を定義し、`Eq` 関数を使って二つの方程式 `eq1` と `eq2` を設定します。そして `solve` 関数を使ってこれらの方程式を同時に解き、`R` (レンガ1個の重さ) を求めます。`evalf()` メソッドは数値解を求めるために使われます。

実際に実行されたコードにより、レンガ1個の重さは約7.8125キログラムであると求めることができました。


#### タスク2：株価情報の入った CSV から時系列グラフをプロットする

In [23]:
request = "添付データを元に、Openの時系列グラフを表示してください"

In [18]:
# Upload a file with an "assistants" purpose
file = client.files.create(
    file=open("Microsoft_Stock.csv", "rb"),
    purpose='assistants'
)

In [20]:
# Create an assistant using the file ID
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model",
    file_ids=[file.id]
)

In [22]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_oNSndcFzuMR7ULMibYv1v8u4', created_at=1708405947, metadata={}, object='thread')


In [ ]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

In [29]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_note1yWzmpJZBsJuAWNUpYrZ",
      "assistant_id": "asst_hQfw4uPXYt7eN9TQkOTrx62H",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-m4C29GGD6hFcOqIHfW2eK1ld"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "時系列グラフを表示しました。グラフには、読み込んだデータセットに含まれる'Open'（開始価格）に関するポイントごとにプロットされています。日付がx軸に、'Open'の価格がy軸に表示されています。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708406060,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_vaNNYFQ3D7qZQ4e6s92bhBqx",
      "thread_id": "thread_oNSndcFzuMR7ULMibYv1v8u4"
    },
    {
      "id": "msg_tGmWJ72MGxTwQOcgUczx5E24",
      "assistant_id": "asst_hQfw4uPXYt7eN9TQkOTrx62H",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "CSVフォーマットでファイルを読み込

In [30]:
data = json.loads(messages.model_dump_json(indent=2))
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)

assistant-m4C29GGD6hFcOqIHfW2eK1ld


In [31]:
content = client.files.content(image_file_id)

image = content.write_to_file("image.png")

In [32]:
from PIL import Image

# Display the image in the default image viewer
image = Image.open("image.png")
image.show()

In [35]:
get_response(client=client, assistant=assistant, thread=thread, request="使用したコードを教えてください")

Status: completed
Elapsed time: 0 minutes 23 seconds
{
  "data": [
    {
      "id": "msg_6Fd93qksMSt2Xjv7pIdG0TYn",
      "assistant_id": "asst_hQfw4uPXYt7eN9TQkOTrx62H",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下がファイルを読み込み、'Open'の時系列グラフを表示するために使用したコードです。\n\n```python\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\n# ファイルを読み込む\nfilename = '/mnt/data/assistant-IJWmhAdARrX6onv5shvF4bMg'\ndf = pd.read_csv(filename)\n\n# 'Date' 列を datetime に変換する\ndf['Date'] = pd.to_datetime(df['Date'])\n\n# 'Open' 時系列グラフをプロットする\nplt.figure(figsize=(14,7))\nplt.plot(df['Date'], df['Open'], marker='o')\nplt.title('Open Price Time Series')\nplt.xlabel('Date')\nplt.ylabel('Open Price')\nplt.grid(True)\nplt.xticks(rotation=45)  # x軸のラベルを見やすくするために回転させる\nplt.tight_layout()  # ラベルが切れないようにする\nplt.show()\n```\n\nこのコードは、CSVファイルを`pandas`を使って読み込み、`matplotlib`を使って時系列データをグラフとしてプロットするために使用されています。プロットには、データフレームの'Date'列がx軸に、'Open'列がy軸に使用されており、各ポイ

In [36]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

{
  "data": [
    {
      "id": "msg_6Fd93qksMSt2Xjv7pIdG0TYn",
      "assistant_id": "asst_hQfw4uPXYt7eN9TQkOTrx62H",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下がファイルを読み込み、'Open'の時系列グラフを表示するために使用したコードです。\n\n```python\nimport pandas as pd\nimport matplotlib.pyplot as plt\n\n# ファイルを読み込む\nfilename = '/mnt/data/assistant-IJWmhAdARrX6onv5shvF4bMg'\ndf = pd.read_csv(filename)\n\n# 'Date' 列を datetime に変換する\ndf['Date'] = pd.to_datetime(df['Date'])\n\n# 'Open' 時系列グラフをプロットする\nplt.figure(figsize=(14,7))\nplt.plot(df['Date'], df['Open'], marker='o')\nplt.title('Open Price Time Series')\nplt.xlabel('Date')\nplt.ylabel('Open Price')\nplt.grid(True)\nplt.xticks(rotation=45)  # x軸のラベルを見やすくするために回転させる\nplt.tight_layout()  # ラベルが切れないようにする\nplt.show()\n```\n\nこのコードは、CSVファイルを`pandas`を使って読み込み、`matplotlib`を使って時系列データをグラフとしてプロットするために使用されています。プロットには、データフレームの'Date'列がx軸に、'Open'列がy軸に使用されており、各ポイントはマーカーで強調されています。また、x軸の日付のラベルは45度傾けて表示されています。"
      

In [37]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

もちろんです。以下がファイルを読み込み、'Open'の時系列グラフを表示するために使用したコードです。

```python
import pandas as pd
import matplotlib.pyplot as plt

# ファイルを読み込む
filename = '/mnt/data/assistant-IJWmhAdARrX6onv5shvF4bMg'
df = pd.read_csv(filename)

# 'Date' 列を datetime に変換する
df['Date'] = pd.to_datetime(df['Date'])

# 'Open' 時系列グラフをプロットする
plt.figure(figsize=(14,7))
plt.plot(df['Date'], df['Open'], marker='o')
plt.title('Open Price Time Series')
plt.xlabel('Date')
plt.ylabel('Open Price')
plt.grid(True)
plt.xticks(rotation=45)  # x軸のラベルを見やすくするために回転させる
plt.tight_layout()  # ラベルが切れないようにする
plt.show()
```

このコードは、CSVファイルを`pandas`を使って読み込み、`matplotlib`を使って時系列データをグラフとしてプロットするために使用されています。プロットには、データフレームの'Date'列がx軸に、'Open'列がy軸に使用されており、各ポイントはマーカーで強調されています。また、x軸の日付のラベルは45度傾けて表示されています。


#### タスク3：CSV を JSON に変換する

In [38]:
request = """
アップロードしたCSVを次のようなJSONに変換して出力してください
# 出力JSONの例
[
  {"Date": "2015-4-1", "Open": 40.6},
  {"Date": "2015-4-2", "Open": 40.66},
  …
]
"""

In [43]:
file = client.files.create(
    file=open("Microsoft_Stock.csv", "rb"),
    purpose='assistants'
)

In [44]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model",
    file_ids=[file.id]
)

In [46]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_UIkCY7knDNhc6WYGDizqWHsP', created_at=1708407509, metadata={}, object='thread')


In [51]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 2 minutes 12 seconds
{
  "data": [
    {
      "id": "msg_iNLdlzSUrRNE01AKCrdUbtHN",
      "assistant_id": "asst_cIlI2FjsDcUAQu6lZD50WZYx",
      "content": [
        {
          "text": {
            "annotations": [
              {
                "end_index": 128,
                "file_path": {
                  "file_id": "assistant-52uTDvBdratFiglFoRx2cRzG"
                },
                "start_index": 95,
                "text": "sandbox:/mnt/data/stock_data.json",
                "type": "file_path"
              }
            ],
            "value": "JSON形式に変換されたデータはファイル `stock_data.json` に保存されました。以下のリンクからダウンロードできます。\n\n[stock_data.json をダウンロード](sandbox:/mnt/data/stock_data.json)\n\nご質問や他のリクエストがありましたら、お知らせください。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708408140,
      "file_ids": [
        "assistant-52uTDvBdratFiglFoRx2cRzG"
      ],
      "metadata": {},
      "object": "thread.message",
      "role": "

In [52]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_iNLdlzSUrRNE01AKCrdUbtHN",
      "assistant_id": "asst_cIlI2FjsDcUAQu6lZD50WZYx",
      "content": [
        {
          "text": {
            "annotations": [
              {
                "end_index": 128,
                "file_path": {
                  "file_id": "assistant-52uTDvBdratFiglFoRx2cRzG"
                },
                "start_index": 95,
                "text": "sandbox:/mnt/data/stock_data.json",
                "type": "file_path"
              }
            ],
            "value": "JSON形式に変換されたデータはファイル `stock_data.json` に保存されました。以下のリンクからダウンロードできます。\n\n[stock_data.json をダウンロード](sandbox:/mnt/data/stock_data.json)\n\nご質問や他のリクエストがありましたら、お知らせください。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708408140,
      "file_ids": [
        "assistant-52uTDvBdratFiglFoRx2cRzG"
      ],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_pUsrq6qKVkLlrpjUlE3V

In [70]:
data = json.loads(messages.model_dump_json(indent=2))
file_id = data["data"][0]["content"][0]["text"]["annotations"][0]["file_path"]["file_id"]

In [71]:
content = client.files.content(file_id)
content_bytes = content.read()

with open("./task3.json", "wb") as file:
    file.write(content_bytes)

In [74]:
get_response(client=client, assistant=assistant, thread=thread, request="使用したコードを教えてください")

Status: completed
Elapsed time: 0 minutes 42 seconds
{
  "data": [
    {
      "id": "msg_kzeRDufnnGKTIfXVrgs4YHMX",
      "assistant_id": "asst_cIlI2FjsDcUAQu6lZD50WZYx",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下に、アップロードされたCSVデータを読み込み、JSON形式に変換し、ファイルに保存するために使用したPythonコードを記載します。\n\n```python\nimport pandas as pd\nimport json\n\n# Load the uploaded CSV file\nfile_path = '/mnt/data/assistant-DffPxNcC2M77mw9W3cCHbUHy'\ndf = pd.read_csv(file_path)\n\n# We only need the 'Date' and 'Open' columns for the JSON output\ndf_json = df[['Date', 'Open']]\n\n# Convert the 'Date' format to 'YYYY-MM-DD' without time\ndf_json['Date'] = pd.to_datetime(df_json['Date']).dt.strftime('%Y-%m-%d')\n\n# Convert the DataFrame to a list of dictionaries\njson_output = df_json.to_dict(orient='records')\n\n# Convert list of dictionaries to JSON format\njson_data = json.dumps(json_output, ensure_ascii=False, indent=2)\n\n# Save the JSON data to a 

In [75]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_kzeRDufnnGKTIfXVrgs4YHMX",
      "assistant_id": "asst_cIlI2FjsDcUAQu6lZD50WZYx",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下に、アップロードされたCSVデータを読み込み、JSON形式に変換し、ファイルに保存するために使用したPythonコードを記載します。\n\n```python\nimport pandas as pd\nimport json\n\n# Load the uploaded CSV file\nfile_path = '/mnt/data/assistant-DffPxNcC2M77mw9W3cCHbUHy'\ndf = pd.read_csv(file_path)\n\n# We only need the 'Date' and 'Open' columns for the JSON output\ndf_json = df[['Date', 'Open']]\n\n# Convert the 'Date' format to 'YYYY-MM-DD' without time\ndf_json['Date'] = pd.to_datetime(df_json['Date']).dt.strftime('%Y-%m-%d')\n\n# Convert the DataFrame to a list of dictionaries\njson_output = df_json.to_dict(orient='records')\n\n# Convert list of dictionaries to JSON format\njson_data = json.dumps(json_output, ensure_ascii=False, indent=2)\n\n# Save the JSON data to a file\njson_file_path = '/mnt/data/stock_data.json'\nw

In [76]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

もちろんです。以下に、アップロードされたCSVデータを読み込み、JSON形式に変換し、ファイルに保存するために使用したPythonコードを記載します。

```python
import pandas as pd
import json

# Load the uploaded CSV file
file_path = '/mnt/data/assistant-DffPxNcC2M77mw9W3cCHbUHy'
df = pd.read_csv(file_path)

# We only need the 'Date' and 'Open' columns for the JSON output
df_json = df[['Date', 'Open']]

# Convert the 'Date' format to 'YYYY-MM-DD' without time
df_json['Date'] = pd.to_datetime(df_json['Date']).dt.strftime('%Y-%m-%d')

# Convert the DataFrame to a list of dictionaries
json_output = df_json.to_dict(orient='records')

# Convert list of dictionaries to JSON format
json_data = json.dumps(json_output, ensure_ascii=False, indent=2)

# Save the JSON data to a file
json_file_path = '/mnt/data/stock_data.json'
with open(json_file_path, 'w', encoding='utf-8') as file:
    file.write(json_data)

# Provide the file path for download
json_file_path
```

このコードは、まず `pandas` ライブラリを使用してCSVファイルを読み込み、必要な列のみを選択し、`strftime` 関数で日付形式を変換します。その後、`to_dict` メソッドでデータフレーム

### 発展的なタスクを解く

#### タスク1 - 発展：EM アルゴリズムを解く

In [4]:
request = """
ふたつのコインA, Bがあります。コイントスをすると、AのほうがBより表が出やすいことが分かっています。どちらかのコインを使って20回コイントスを行うという試行を10回繰り返した結果、表が出た回数は次のようになりました。A、Bそれぞれのコインの表が出る確率をEMアルゴリズムで推定してください。
表が出た回数	15	4	6	13	3	5	4	4	7	2
"""

In [5]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model"
)

In [6]:
thread = client.beta.threads.create()

In [7]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 3 minutes 56 seconds
{
  "data": [
    {
      "id": "msg_0035aBecFiZRkXqyLqZuJtQ1",
      "assistant_id": "asst_4BKrPWfAgZxbfumpy5n9fMOi",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "EMアルゴリズムに基づく推定結果によると、コインAの表が出る確率は約0.2185（約21.85%）、コインBの表が出る確率は約0.6970（約69.70%）であることが示されました。\n\nEMアルゴリズムの性質上、初期値によって最終結果が若干異なることがあります。したがって、この推定結果はその実行の一例であり、異なるランダムシードを使用すると異なる推定が得られる可能性があります。ただし、推定された確率がAの方がBよりも明らかに小さいことから、初期の記述に合致していることがわかります。\n\n対数尤度の最終値は約-25.279です。この尤度は推定されたモデルがデータをどれくらいよく説明しているかの指標であり、EMアルゴリズムはこの尤度を最大化するようなパラメータを探索します。尤度が最大となる地点（または収束判定基準に十分近い地点）でアルゴリズムが停止します。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708496892,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_l2xoDRaL4YybLvO7yamnPWd8",
      "thread_id": "thread_wLveoOoQremQTeAyu3xTD0rH"
    },
    {
      "id": "msg_a6eTDQkAG3MaaDP

In [8]:
get_response(client=client, assistant=assistant, thread=thread, request="使用したコードを教えてください")

Status: completed
Elapsed time: 1 minutes 54 seconds
{
  "data": [
    {
      "id": "msg_uXTQfprFBaKjm5P4dogOUA6w",
      "assistant_id": "asst_4BKrPWfAgZxbfumpy5n9fMOi",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下はコインAとコインBの表が出る確率を推定するために使用したEMアルゴリズムのコードです。このコードはコイントスの結果を受け取り、両コインの確率を推定します。\n\n```python\nimport numpy as np\nfrom scipy.stats import binom\n\ndef em_algorithm(tosses, tol=1e-6, max_iter=10000):\n    # 初期パラメータpの推定をランダムにセット\n    p = np.array([np.random.rand(), np.random.rand()])\n    \n    # 変数の初期化\n    n = len(tosses)  # トスの回数\n    heads = np.array(tosses)  # 表が出た回数\n    num_tosses = 20  # それぞれの試行でのトスの回数\n    \n    log_likelihood = -np.inf\n    \n    # EMアルゴリズムの実行\n    for _ in range(max_iter):\n        # E-step: 各トライアルのコインA、Bからそれぞれのデータが来たとする確率の割合を計算\n        w_a = binom.pmf(heads, num_tosses, p[0]) * 0.5  # コインAによる寄与(等確率を仮定)\n        w_b = binom.pmf(heads, num_tosses, p[1]) * 0.5  # コインBによる寄与(等確率を仮定)\n  

In [9]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

{
  "data": [
    {
      "id": "msg_uXTQfprFBaKjm5P4dogOUA6w",
      "assistant_id": "asst_4BKrPWfAgZxbfumpy5n9fMOi",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下はコインAとコインBの表が出る確率を推定するために使用したEMアルゴリズムのコードです。このコードはコイントスの結果を受け取り、両コインの確率を推定します。\n\n```python\nimport numpy as np\nfrom scipy.stats import binom\n\ndef em_algorithm(tosses, tol=1e-6, max_iter=10000):\n    # 初期パラメータpの推定をランダムにセット\n    p = np.array([np.random.rand(), np.random.rand()])\n    \n    # 変数の初期化\n    n = len(tosses)  # トスの回数\n    heads = np.array(tosses)  # 表が出た回数\n    num_tosses = 20  # それぞれの試行でのトスの回数\n    \n    log_likelihood = -np.inf\n    \n    # EMアルゴリズムの実行\n    for _ in range(max_iter):\n        # E-step: 各トライアルのコインA、Bからそれぞれのデータが来たとする確率の割合を計算\n        w_a = binom.pmf(heads, num_tosses, p[0]) * 0.5  # コインAによる寄与(等確率を仮定)\n        w_b = binom.pmf(heads, num_tosses, p[1]) * 0.5  # コインBによる寄与(等確率を仮定)\n        w_total = w_a + w_b\n        w_a /= w_total\n  

In [10]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

もちろんです。以下はコインAとコインBの表が出る確率を推定するために使用したEMアルゴリズムのコードです。このコードはコイントスの結果を受け取り、両コインの確率を推定します。

```python
import numpy as np
from scipy.stats import binom

def em_algorithm(tosses, tol=1e-6, max_iter=10000):
    # 初期パラメータpの推定をランダムにセット
    p = np.array([np.random.rand(), np.random.rand()])
    
    # 変数の初期化
    n = len(tosses)  # トスの回数
    heads = np.array(tosses)  # 表が出た回数
    num_tosses = 20  # それぞれの試行でのトスの回数
    
    log_likelihood = -np.inf
    
    # EMアルゴリズムの実行
    for _ in range(max_iter):
        # E-step: 各トライアルのコインA、Bからそれぞれのデータが来たとする確率の割合を計算
        w_a = binom.pmf(heads, num_tosses, p[0]) * 0.5  # コインAによる寄与(等確率を仮定)
        w_b = binom.pmf(heads, num_tosses, p[1]) * 0.5  # コインBによる寄与(等確率を仮定)
        w_total = w_a + w_b
        w_a /= w_total
        w_b /= w_total
        
        # M-step: 新しいパラメータの推定値を計算
        p_new = np.array([
            np.sum(w_a * heads) / np.sum(w_a * num_tosses),
            np.sum(w_b * heads) / np.sum(w_b * num_tosses)
        ])
        
        # 対数尤度の

#### タスク2 - 発展：時系列予測結果を可視化する

In [11]:
request = "添付データを元に、2019年までのOpenの値から予測モデルを構築し、予測した2020年以降のOpenの値を実際の値と合わせてグラフに表示してください"

In [12]:
file = client.files.create(
    file=open("Microsoft_Stock.csv", "rb"),
    purpose='assistants'
)

In [13]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model",
    file_ids=[file.id]
)

In [14]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_4FCewxolkWnxhqkci14Jx0ly', created_at=1708498111, metadata={}, object='thread')


In [15]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 2 minutes 22 seconds
{
  "data": [
    {
      "id": "msg_ssiM4JE4D1ZR7YKvKRMGyuKQ",
      "assistant_id": "asst_aqv7LILmkJt9ZQU8KaALQ4UO",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-86UzzaDx4CnLndC4si3mhTTz"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "グラフには、2019年までの実際の「Open」の値（青色）、2020年以降の実際の「Open」の値（緑色）、そして予測された「Open」の値（赤色、破線）が表示されています。線形回帰モデルを用いると、長期的な傾向を捉えることができますが、短期的な変動や特定のパターンは捉えられません。\n\nこのモデルは単純なので、実際のデータの複雑な挙動に完全には対応していません。より詳細な分析や予測を行うためには、時系列分析専用のモデルやディープラーニングアプローチを用いることが考えられます。もし必要であれば、より洗練された時系列モデルを構築する手助けもできます。ご希望であればお知らせください。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708498229,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_oEbTr086B2gy6Y6Q0xpOpaVS",
      "thread_id": "thread_4FCewxolkW

In [19]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

{
  "data": [
    {
      "id": "msg_ssiM4JE4D1ZR7YKvKRMGyuKQ",
      "assistant_id": "asst_aqv7LILmkJt9ZQU8KaALQ4UO",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-86UzzaDx4CnLndC4si3mhTTz"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "グラフには、2019年までの実際の「Open」の値（青色）、2020年以降の実際の「Open」の値（緑色）、そして予測された「Open」の値（赤色、破線）が表示されています。線形回帰モデルを用いると、長期的な傾向を捉えることができますが、短期的な変動や特定のパターンは捉えられません。\n\nこのモデルは単純なので、実際のデータの複雑な挙動に完全には対応していません。より詳細な分析や予測を行うためには、時系列分析専用のモデルやディープラーニングアプローチを用いることが考えられます。もし必要であれば、より洗練された時系列モデルを構築する手助けもできます。ご希望であればお知らせください。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708498229,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_oEbTr086B2gy6Y6Q0xpOpaVS",
      "thread_id": "thread_4FCewxolkWnxhqkci14Jx0ly"
    },
    {
      "id": "msg_UupjlWZ

In [20]:
data = json.loads(messages.model_dump_json(indent=2))
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)

assistant-86UzzaDx4CnLndC4si3mhTTz


In [21]:
content = client.files.content(image_file_id)

image = content.write_to_file("image.png")

In [22]:
from PIL import Image

# Display the image in the default image viewer
image = Image.open("image.png")
image.show()

In [23]:
get_response(client=client, assistant=assistant, thread=thread, request="使用したコードを教えてください")

Status: completed
Elapsed time: 1 minutes 4 seconds
{
  "data": [
    {
      "id": "msg_irAg9ldhvWV6PNdRBBaIqIJu",
      "assistant_id": "asst_aqv7LILmkJt9ZQU8KaALQ4UO",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下は、アップロードされたデータを読み込んで線形回帰モデルを構築し、2019年までの「Open」の値から予測を行い、その結果をグラフに表示するために使用したコードです。\n\n```python\nimport pandas as pd\nfrom sklearn.linear_model import LinearRegression\nimport matplotlib.pyplot as plt\nimport numpy as np\n\n# Load the uploaded file\nfile_path = '/mnt/data/assistant-EmOKijP71DrqLQJI2mi5v1RA'\ndata = pd.read_csv(file_path)\n\n# Convert 'Date' to datetime\ndata['Date'] = pd.to_datetime(data['Date'])\n\n# Sort data by date just in case\ndata.sort_values('Date', inplace=True)\n\n# Filter data up to the end of 2019\ndata_train = data[data['Date'].dt.year <= 2019]\n\n# Prepare training data\nX_train = np.array(range(len(data_train))).reshape(-1, 1)  # We use days as numbers for simplicity\ny_train =

In [24]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

{
  "data": [
    {
      "id": "msg_irAg9ldhvWV6PNdRBBaIqIJu",
      "assistant_id": "asst_aqv7LILmkJt9ZQU8KaALQ4UO",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "もちろんです。以下は、アップロードされたデータを読み込んで線形回帰モデルを構築し、2019年までの「Open」の値から予測を行い、その結果をグラフに表示するために使用したコードです。\n\n```python\nimport pandas as pd\nfrom sklearn.linear_model import LinearRegression\nimport matplotlib.pyplot as plt\nimport numpy as np\n\n# Load the uploaded file\nfile_path = '/mnt/data/assistant-EmOKijP71DrqLQJI2mi5v1RA'\ndata = pd.read_csv(file_path)\n\n# Convert 'Date' to datetime\ndata['Date'] = pd.to_datetime(data['Date'])\n\n# Sort data by date just in case\ndata.sort_values('Date', inplace=True)\n\n# Filter data up to the end of 2019\ndata_train = data[data['Date'].dt.year <= 2019]\n\n# Prepare training data\nX_train = np.array(range(len(data_train))).reshape(-1, 1)  # We use days as numbers for simplicity\ny_train = data_train['Open'].values\n\n# Create and train the

In [25]:
data = json.loads(messages.model_dump_json(indent=2))
code = data['data'][0]['content'][0]['text']['value']
print(code)

もちろんです。以下は、アップロードされたデータを読み込んで線形回帰モデルを構築し、2019年までの「Open」の値から予測を行い、その結果をグラフに表示するために使用したコードです。

```python
import pandas as pd
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np

# Load the uploaded file
file_path = '/mnt/data/assistant-EmOKijP71DrqLQJI2mi5v1RA'
data = pd.read_csv(file_path)

# Convert 'Date' to datetime
data['Date'] = pd.to_datetime(data['Date'])

# Sort data by date just in case
data.sort_values('Date', inplace=True)

# Filter data up to the end of 2019
data_train = data[data['Date'].dt.year <= 2019]

# Prepare training data
X_train = np.array(range(len(data_train))).reshape(-1, 1)  # We use days as numbers for simplicity
y_train = data_train['Open'].values

# Create and train the Linear Regression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict the Open values using the model
data['Predicted_Open'] = model.predict(np.array(range(len(data))).reshape(-1, 1))

# Split the dataframe into training+vali

#### タスク3 - 発展：CSV を月毎に平均を取ってから JSON にする

In [26]:
request = """
アップロードしたCSVを次のような月毎に平均を取ったJSONに変換して出力してください
# 出力JSONの例
[
  {"Date": "2015-4", "Open": 40.6},
  {"Date": "2015-5", "Open": 40.66},
  …
]
"""

In [27]:
file = client.files.create(
    file=open("Microsoft_Stock.csv", "rb"),
    purpose='assistants'
)

In [28]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model",
    file_ids=[file.id]
)

In [29]:
thread = client.beta.threads.create()

In [30]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 1 minutes 27 seconds
{
  "data": [
    {
      "id": "msg_3Be65NWza3MjgsNKbMH5TSEm",
      "assistant_id": "asst_vLzqB7KJARK2SZm0xlOXFqlM",
      "content": [
        {
          "text": {
            "annotations": [
              {
                "end_index": 141,
                "file_path": {
                  "file_id": "assistant-Gk7jMzrWVcrg8gSTn0mXTkkV"
                },
                "start_index": 95,
                "text": "sandbox:/mnt/data/monthly_avg_open_prices.json",
                "type": "file_path"
              }
            ],
            "value": "月毎の「Open」平均値を含むJSONデータファイルが生成されました。以下のリンクからダウンロードできます：\n\n[ダウンロード: monthly_avg_open_prices.json](sandbox:/mnt/data/monthly_avg_open_prices.json)"
          },
          "type": "text"
        }
      ],
      "created_at": 1708499255,
      "file_ids": [
        "assistant-Gk7jMzrWVcrg8gSTn0mXTkkV"
      ],
      "metadata": {},
      "object": "thread.message",
      "role": "assist

In [31]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

{
  "data": [
    {
      "id": "msg_3Be65NWza3MjgsNKbMH5TSEm",
      "assistant_id": "asst_vLzqB7KJARK2SZm0xlOXFqlM",
      "content": [
        {
          "text": {
            "annotations": [
              {
                "end_index": 141,
                "file_path": {
                  "file_id": "assistant-Gk7jMzrWVcrg8gSTn0mXTkkV"
                },
                "start_index": 95,
                "text": "sandbox:/mnt/data/monthly_avg_open_prices.json",
                "type": "file_path"
              }
            ],
            "value": "月毎の「Open」平均値を含むJSONデータファイルが生成されました。以下のリンクからダウンロードできます：\n\n[ダウンロード: monthly_avg_open_prices.json](sandbox:/mnt/data/monthly_avg_open_prices.json)"
          },
          "type": "text"
        }
      ],
      "created_at": 1708499255,
      "file_ids": [
        "assistant-Gk7jMzrWVcrg8gSTn0mXTkkV"
      ],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_1Sy91HKPVFiVK2yRF2JhZaMY",

In [32]:
data = json.loads(messages.model_dump_json(indent=2))
file_id = data["data"][0]["content"][0]["text"]["annotations"][0]["file_path"]["file_id"]

In [33]:
content = client.files.content(file_id)
content_bytes = content.read()

with open("./task3_progress.json", "wb") as file:
    file.write(content_bytes)

### 拡張性について検討

#### できないことの確認

In [5]:
request = "IBMの株価を1991年から2023年までmatplotlibでplotしてください"

In [6]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model"
)

In [7]:
thread = client.beta.threads.create()

In [8]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 0 minutes 22 seconds
{
  "data": [
    {
      "id": "msg_at0qYL1jmDaz5q4fG1EHoYqS",
      "assistant_id": "asst_JgGHGeM3NGhOHygUh4tPQJ3n",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "申し訳ありませんが、この環境にはインターネットアクセスが制限されているため、直接株の価格データを取得することはできません。ただし、データを持っている場合は、それをアップロードしてからグラフを作成できます。お持ちのデータをアップロードして指示をしていただければ、それに基づいてIBMの株価をプロットすることができます。データがCSVやExcelファイルであれば、それを解析してグラフにすることが可能です。"
          },
          "type": "text"
        }
      ],
      "created_at": 1708580299,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_lnb2C6St2Wisry4hbK9COEhg",
      "thread_id": "thread_OCZ0MZDJdL1pdyBisUy2UetC"
    },
    {
      "id": "msg_O4NCSc0j4BgMIcqAaevXSuhC",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "IBMの株価を1991年から2023年までmatplotlibでplotしてくださ

申し訳ありませんが、この環境にはインターネットアクセスが制限されているため、直接株の価格データを取得することはできません。ただし、データを持っている場合は、それをアップロードしてからグラフを作成できます。お持ちのデータをアップロードして指示をしていただければ、それに基づいてIBMの株価をプロットすることができます。データがCSVやExcelファイルであれば、それを解析してグラフにすることが可能です。

#### 対策の検討

##### Web 検索結果の `snippet` を使用

In [47]:
request = "IBMの株価を1991年から2023年までmatplotlibでplotしてください"

In [48]:
subscription_key = credentials["web_search"]["key"]

In [49]:
search_url = "https://api.bing.microsoft.com/v7.0/search"

In [50]:
search_term = "IBM 株価"

In [51]:
import requests

In [52]:
headers = {"Ocp-Apim-Subscription-Key": subscription_key}
params = {"q": search_term, "textDecorations": True, "textFormat": "HTML"}
response = requests.get(search_url, headers=headers, params=params)
response.raise_for_status()
search_results = response.json()

In [53]:
from IPython.display import HTML

In [54]:
rows = "\n".join(["""<tr>
                       <td><a href=\"{0}\">{1}</a></td>
                       <td>{2}</td>
                     </tr>""".format(v["url"], v["name"], v["snippet"])
                  for v in search_results["webPages"]["value"]])
HTML("<table>{0}</table>".format(rows))

インターナショナル・ビジネス・マシーンズ【IBM】：株価 ...,120.55 ( 23/05/11) インターナショナル・ビジネス・マシーンズ【IBM】の株式チャート推移をご覧いただけます。. 前日終値、高値、安値はもちろんのこと出来高や売買代金もご覧いただけます。. Yahoo!ファイナンスでは株価速報、チャート、ランキング ...
IBM 株価[IBM/I B M]最新ニュース 日経会社情報DIGITAL ...,IBM 株価 [IBM/I B M]最新ニュース 日経会社情報DIGITAL - 日本経済新聞. 米国株： IBM NYSE. 1911年創業の「C-T-R」を前身とする老舗IT企業。 IBM （International Business Machines） 188.20＄ ＋3.17(＋1.71％) 現在値 (3/1 16:00) / 前日比. 188.20＄...
IBM【IBM】の株価/株式・基本情報 - 株探(かぶたん),IBM【IBM】の株価/株式・基本情報. NYSE. 株価 15分ディレイ. リアルタイムに変更. IBM. $188.20. 前日比. +3.17 (+1.71 %) NY時間. 01日 16:00. 日本時間. 02日 06:00. $188.20. 0 (0.00%) 01日 19:59. 02日 09:59. PER. 23.1 倍. PBR. 7.63 倍....
インターナショナル・ビジネス・マシーンズ【IBM】：株価 ...,"IBM IT・通信. 15分ディレイ株価. 3/1 10:04. （現地時刻: 2/29 20:04） ポートフォリオに追加. 前日比. -0.27(-0.15 %) 詳細情報. チャート. 時系列. 企業情報. 業績. 掲示板. 米国株トップ. インターナショナル・ビジネス・マシーンズの株価チャート. 指標を表示. 前日終値 02/28. 185.30. 始値 02/29. 186.15. 時価総額 02/29. 167,115,461.00 千ドル. 発行済株式数 03/01. 903,180,353 株. 高値 02/29. 186.85. 安値 02/29. 184.69. PER（予想） 02/29. (連) 22.73 倍. EPS（予想） 2023/12."
IBM:New York 株価 - IBM - Bloomberg Markets,IBM (IBM:New York) の株価、株式情報、チャート、関連ニュースなど、企業概要や株価の分析をご覧いただけます。
IBM株価、10年超ぶり高値 予想上回る通期増収率見通し受け ...,IBM株価は一時、2013年6月以来の高値となる196．58ドルまで上昇。 時価総額は約200億ドル増加した。 年初来では約20％上昇。 1日当たりの上昇率では20年3月以降で最大となる見込み。 今回の発表を受けて少なくとも8人のアナリストが目標株価を引き上げた。...
IBMの株価とチャート — NYSE:IBM — TradingView,"International Business Machines Corporation. IBMNew York Stock Exchange. 188.20USD. +3.17 +1.71% 3月 1, 16:59 UTC (協定世界時)-8 にクローズ. スーパーチャートで確認. 概要. ファンダメンタル ニュース アイデア テクニカル 予測. IBM チャート. 今日 1.71% 5日 1.40% 1ヶ月 0.61% 6ヶ月 27.80% 年初来 15.58% 1年 46.58% 5年 40.75% 全期間 1184.66% 主要統計. 時価総額. ‪172.53 B‬ USD. 配当利回り (予測値) 3.53% PER: 株価収益率 (直近12ヶ月) 22.78."
IBM - | Stock Price & Latest News | Reuters,"102.23. 発行済株式（MIL） 916.74. 時価総額. 169,872.80. 予想株価収益率. 18.36. 配当利回り. 3.58. 重要な統計. 2.57平均評価 -アナリスト21人. 売却. 保有. 購入. 特別項目を除く株価収益率（仲値）"
業務・財務情報 IBM[IBM/I B M]日経会社情報DIGITAL - 日本 ...,IBM （International Business Machines） 188.20＄ ＋3.17(＋1.71％) 現在値 (3/1 16:00) / 前日比. 188.20＄ ±0.00(±0.00％) アフター・マーケット (3/1 19:22) / 当日終値比 （解説） トップ. ニュース. チャート. 株価. 業績・財務. 開示情報【EDGAR】 通期 四半期....
チャート IBM[IBM/I B M]日経会社情報DIGITAL - 日本経済新聞,"IBM （International Business Machines） 現在値 (2/28 11:54)： 183.95 ＄. 前日比： －0.92 (－0.49％) トップ. ニュース. チャート. 株価. 業績・財務. 開示情報【EDGAR】 (ドル) (万株) 2/1 1/1 12/1 150 160 170 180 190 200 0 1,000 2,000 3,000...."


In [55]:
def get_reference(response: requests.models.Response) -> str:
    search_results = response.json()
    reference = ""
    for v in search_results["webPages"]["value"]:
        print(v["snippet"])
        reference += v["snippet"]
        reference += "\n"
    return reference

In [56]:
reference = get_reference(response)

120.55 ( 23/05/11) インターナショナル・ビジネス・マシーンズ【IBM】の株式チャート推移をご覧いただけます。. 前日終値、高値、安値はもちろんのこと出来高や売買代金もご覧いただけます。. Yahoo!ファイナンスでは<b>株価</b>速報、チャート、ランキング ...
IBM <b>株価</b> [IBM/I B M]最新ニュース 日経会社情報DIGITAL - 日本経済新聞. 米国株： IBM NYSE. 1911年創業の「C-T-R」を前身とする老舗IT企業。 IBM （International Business Machines） 188.20＄ ＋3.17(＋1.71％) 現在値 (3/1 16:00) / 前日比. 188.20＄...
IBM【IBM】の<b>株価</b>/株式・基本情報. NYSE. <b>株価</b> 15分ディレイ. リアルタイムに変更. IBM. $188.20. 前日比. +3.17 (+1.71 %) NY時間. 01日 16:00. 日本時間. 02日 06:00. $188.20. 0 (0.00%) 01日 19:59. 02日 09:59. PER. 23.1 倍. PBR. 7.63 倍....
IBM IT・通信. 15分ディレイ<b>株価</b>. 3/1 10:04. （現地時刻: 2/29 20:04） ポートフォリオに追加. 前日比. -0.27(-0.15 %) 詳細情報. チャート. 時系列. 企業情報. 業績. 掲示板. 米国株トップ. インターナショナル・ビジネス・マシーンズの<b>株価</b>チャート. 指標を表示. 前日終値 02/28. 185.30. 始値 02/29. 186.15. 時価総額 02/29. 167,115,461.00 千ドル. 発行済株式数 03/01. 903,180,353 株. 高値 02/29. 186.85. 安値 02/29. 184.69. PER（予想） 02/29. (連) 22.73 倍. EPS（予想） 2023/12.
IBM (IBM:New York) の<b>株価</b>、株式情報、チャート、関連ニュースなど、企業概要や<b>株価</b>の分析をご覧いただけます。
<b>IBM株価</b>は一時、2013年6月以来の高値

In [57]:
with open("snippet_reference.txt", "w", encoding="utf-8") as f:
    f.write(reference)

![](snippet_tokens.PNG)

In [58]:
print(1622 / 1000 * 4.513, "円")

7.320086000000001 円


In [13]:
modified_content = "必要に応じてcontextの情報を参考にし、以下の質問に答えてください。\n" + request + "\n### context ###\n" + reference

In [31]:
print(modified_content)

必要に応じてcontextの情報を参考にし、以下の質問に答えてください。
IBMの株価を1991年から2023年までmatplotlibでplotしてください
### context ###
120.55 ( 23/05/11) インターナショナル・ビジネス・マシーンズ【IBM】の株式チャート推移をご覧いただけます。. 前日終値、高値、安値はもちろんのこと出来高や売買代金もご覧いただけます。. Yahoo!ファイナンスでは<b>株価</b>速報、チャート、ランキング ...
IBM （International Business Machines） (＋0.23％) 現在値 (2/28 16:00) / 前日比. トップ. ニュース. チャート. <b>株価</b>. 業績・財務. 開示情報【EDGAR】 始値 (09:30) 184.63＄. 高値 (15:59) 185.37＄. 安値 (09:58) 183.55＄. 売買高 （解説） 3,214,547 株....
IBM【IBM】の<b>株価</b>/株式・基本情報. NYSE. <b>株価</b> 15分ディレイ. リアルタイムに変更. IBM. $184.13. 前日比. -1.59 (-0.86 %) NY時間. 26日 16:00. 日本時間. 27日 06:00. $184.00. -0.13 (-0.07 %) 26日 19:59. 27日 09:59. PER. 102 倍. PBR. 7.58...
IBM (IBM:New York) の<b>株価</b>、株式情報、チャート、関連ニュースなど、企業概要や<b>株価</b>の分析をご覧いただけます。
<b>IBM株価</b>は一時、2013年6月以来の高値となる196．58ドルまで上昇。 時価総額は約200億ドル増加した。 年初来では約20％上昇。 1日当たりの上昇率では20年3月以降で最大となる見込み。 今回の発表を受けて少なくとも8人のアナリストが目標<b>株価</b>を引き上げた。...
最新日時：2024-02-25 07:02 - IBM (IBM)の<b>株価</b>分析結果。. AI<b>株価</b>診断。. 現在は【売り】 (理論<b>株価</b> 164.03)と判断されております。. <b>株価</b>、チャート、企業情

In [22]:
client = AzureOpenAI(
    api_key=credentials["openai_gpt4"]["key"],
    api_version=credentials["openai_gpt4"]["version"],
    azure_endpoint=credentials["openai_gpt4"]["endpoint"]
)

In [23]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model"
)

In [24]:
thread = client.beta.threads.create()

In [25]:
get_response(client=client, assistant=assistant, thread=thread, request=modified_content)

Status: completed
Elapsed time: 1 minutes 13 seconds
{
  "data": [
    {
      "id": "msg_q4O6Lzfgji94DPT2RKa8qIV6",
      "assistant_id": "asst_97Na93LFW1DEr7hfB6dbq8i4",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-rFikQcjmSZS2tjYjCw38lla9"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is a simulated plot of IBM's stock price from 1991 to 2023. Please note that this data is purely synthetic and does not represent the actual historical stock prices of IBM. To analyze real historical stock data, one would typically use authentic financial data sources."
          },
          "type": "text"
        }
      ],
      "created_at": 1709279165,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_pcJv6saHttVRN6s9yxJrZA4Y",
      "thread_id": "thread_mlXkjzy8uGc7fpcDl20raKJ3"
    

In [26]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

In [27]:
data = json.loads(messages.model_dump_json(indent=2))
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)

assistant-rFikQcjmSZS2tjYjCw38lla9


In [28]:
content = client.files.content(image_file_id)

image = content.write_to_file("image.png")

![The IBM stock chart from 1991 to 2023 was generated using a code interpreter with web information.](ibm_stock_assistant.png)

!["The actual IBM stock chart from 1991 to 2023."](ibm_stock_actual.png)

Reference:
https://www.macrotrends.net/stocks/charts/IBM/ibm/stock-price-histor


In [29]:
get_response(client=client, assistant=assistant, thread=thread, request="使用したコードをファイルに出力してください。")

Status: completed
Elapsed time: 0 minutes 51 seconds
{
  "data": [
    {
      "id": "msg_nL2AEReSIHUo80OGGJPPcAXO",
      "assistant_id": "asst_97Na93LFW1DEr7hfB6dbq8i4",
      "content": [
        {
          "text": {
            "annotations": [
              {
                "end_index": 265,
                "file_path": {
                  "file_id": "assistant-jhWho7dfdyx8g2rCVT081I9z"
                },
                "start_index": 218,
                "text": "sandbox:/mnt/data/ibm_stock_price_simulation.py",
                "type": "file_path"
              }
            ],
            "value": "The code used to generate the simulated plot of IBM's stock price has been saved to a file named `ibm_stock_price_simulation.py`. You can download it using the following link:\n\n[Download ibm_stock_price_simulation.py](sandbox:/mnt/data/ibm_stock_price_simulation.py)"
          },
          "type": "text"
        }
      ],
      "created_at": 1709280699,
      "file_ids": [
     

In [30]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

data = json.loads(messages.model_dump_json(indent=2))
file_id = data["data"][0]["content"][0]["text"]["annotations"][0]["file_path"]["file_id"]

content = client.files.content(file_id)
content_bytes = content.read()

with open("./ibm_stock_price_simulation.py", "wb") as file:
    file.write(content_bytes)

```python

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Generate a date range from 1991 to 2023
date_range = pd.date_range(start="1991-01-01", end="2023-12-31", freq='B')  # 'B' stands for business day frequency

# Create a synthetic stock price series using random walk
np.random.seed(42)  # For reproducibility
stock_prices = np.round(100 + np.random.randn(len(date_range)).cumsum(), 2)

# Put the dates and stock prices into a DataFrame
ibm_stock_prices = pd.DataFrame({'Date': date_range, 'Stock_Price': stock_prices})

# Set the date as the index
ibm_stock_prices.set_index('Date', inplace=True)

# Plot the data
plt.figure(figsize=(14, 7))
plt.plot(ibm_stock_prices.index, ibm_stock_prices['Stock_Price'], label='IBM Stock Price')
plt.title('IBM Stock Price (1991 - 2023)')
plt.xlabel('Year')
plt.ylabel('Stock Price (USD)')
plt.legend()
plt.grid(True)
plt.show()
```

##### Web 検索結果の `url` から `get` した HTML を使用

In [31]:
def get_full_html_reference(response: requests.models.Response) -> str:
    search_results = response.json()
    reference = ""
    for v in search_results["webPages"]["value"]:
        r = requests.get(v["url"])
        reference += r.text
        reference += "\n"
    return reference

In [32]:
full_html_reference = get_full_html_reference(response)

![](full_tokens.png)

Reference:
- https://platform.openai.com/tokenizer
- https://azure.microsoft.com/en-us/pricing/details/cognitive-services/openai-service/
- https://learn.microsoft.com/en-us/azure/ai-services/openai/quotas-limits

In [36]:
print(690285 / 1000 * 4.513, "円")

3115.2562049999997 円


##### Web 検索結果の `url` から `get` した HTML のテキストを使用

In [38]:
import html2text

In [42]:
def get_full_text_reference(response: requests.models.Response) -> str:
    search_results = response.json()
    reference = ""
    for v in search_results["webPages"]["value"]:
        r = requests.get(v["url"])
        reference += html2text.html2text(r.text)
        reference += "\n"
    return reference

In [43]:
full_text_reference = get_full_text_reference(response)

In [45]:
with open("full_text_reference.txt", "w", encoding="utf-8") as f:
    f.write(full_text_reference)

![](full_text_tokens.PNG)

In [46]:
print(75767 / 1000 * 4.513, "円")

341.936471 円


##### アップロードファイル経由で情報を渡せるか確認

###### 質問に答えられないことを確認

In [94]:
request = "ノーベル賞の最新の受賞者は誰ですか？"

In [96]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model-a"
)

In [97]:
thread = client.beta.threads.create()

In [98]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 0 minutes 17 seconds
{
  "data": [
    {
      "id": "msg_ZH1ijOrjrARbYBmvphQLx0Xk",
      "assistant_id": "asst_zAgJ8heqRFikbJCqPsRVAMHw",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "申し訳ありませんが、インターネットへのアクセスが制限されているため、ノーベル賞の最新の受賞者に関する情報をリアルタイムで提供することができません。ノーベル賞の最新の受賞者については、ノーベル賞の公式ウェブサイトまたは信頼できるニュースソースで最新の情報を確認してください。また、2022年のノーベル賞の受賞者についての情報を提供することは可能ですが、これがあなたの質問の対象かどうかは明確ではありません。追加の情報が必要であれば教えてください。"
          },
          "type": "text"
        }
      ],
      "created_at": 1709541154,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_X4bSLH30hrXpIf3BxzJga8kH",
      "thread_id": "thread_h7p1zUHpkmnVkKuw6jddiNNF"
    },
    {
      "id": "msg_CaprINbheoMfC3bSh7PAwdcb",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "ノーベル賞の最新の受賞者は誰ですか？"


申し訳ありませんが、私の情報は2023年4月までのものであり、その後の更新がありません。2023年のノーベル賞受賞者についてはインターネットで検索するか、関連する公式ウェブサイトを確認して最新の情報を入手する必要があります。ノーベル賞は毎年10月に発表され、12月に授賞式が行われるため、最新の情報が必要な場合はその時期に確認することをお勧めします。

###### アップロードファイルで情報を渡して質問

In [99]:
request = "添付データを元に、次の質問に答えてください。ノーベル賞の最新の受賞者は誰ですか？"

In [100]:
file = client.files.create(
    file=open("nobel_prize_info.txt", "rb"),
    purpose='assistants'
)

In [101]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model-b",
    file_ids=[file.id]
)

In [102]:
thread = client.beta.threads.create()

In [103]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 4 minutes 8 seconds
{
  "data": [
    {
      "id": "msg_eHqAW7wlUZECrk0ntte1FdVP",
      "assistant_id": "asst_JrZLqiyaMGXpqZT4vbTjGmf9",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "エクセルのデータを調べたところ、以下の通り2023年のノーベル賞の受賞者についての情報が見つかりました：\n\n- ノーベル経済学賞：ハーバード大学のゴールディン氏（特定のフルネームはデータに含まれていない）\n- ノーベル平和賞：イランの人権活動家 ナルゲス・モハンマディ氏\n- ノーベル文学賞：ノルウェーの劇作家 ヨン・フォッセ氏\n- ノーベル化学賞：「量子ドット」を発見した研究者ら3人（特定の名前はデータに含まれていない）\n- ノーベル物理学賞：「アト秒」で光を出す手法を開発した米欧の研究者3人（特定の名前はデータに含まれていない）\n- ノーベル生理学・医学賞：新型コロナのワクチン開発に貢献したカタリン・カリコ氏ら2人（もう一人の名前はデータに含まれていない）\n\n各賞の発表日が新しい順に記載されていると仮定すると、最新の受賞者はノーベル経済学賞のハーバード大学のゴールディン氏のようです。\n\nただし、正確な最新の受賞者のフルネームや他の受賞者の詳細な情報は提供されたデータに含まれていません。フルネームを知りたければ、他の信頼できる情報源を検索する必要があります。"
          },
          "type": "text"
        }
      ],
      "created_at": 1709541376,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_AQuJmd1DbgKGk

ノーベル賞のカテゴリーごとに2023年の最新の受賞者は次のとおりです：\n\n- ノーベル経済学賞：ハーバード大学のゴールディン教授 (更新日: 2023年10月9日)\n- ノーベル平和賞：イランの人権活動家 ナルゲス・モハンマディ氏 (更新日: 2023年10月6日)\n- ノーベル文学賞：ノルウェーの劇作家 ヨン・フォッセ氏 (更新日: 2023年10月5日)\n- ノーベル化学賞：「量子ドット」を発見した研究者ら3人 (更新日: 2023年10月4日)\n- ノーベル物理学賞：「アト秒」で光を出す手法を開発した米欧の研究者3人 (更新日: 2023年10月3日)\n- ノーベル生理学・医学賞：新型コロナのワクチン開発に貢献したカリコ氏ら2人 (更新日: 2023年10月2日)\n\n最新の受賞日を基準にすると、最新の受賞者はハーバード大学のゴールディン教授で、ノーベル経済学賞を受賞されています。

###### プロンプトで情報を渡して質問

In [104]:
with open("nobel_prize_info.txt", "r", encoding="utf-8") as f:
    nobel_prize_info = f.readlines()

In [105]:
request = "以下の情報を元に、次の質問に答えてください。ノーベル賞の最新の受賞者は誰ですか？" + "\n# 情報\n" + "".join(nobel_prize_info)

In [106]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model-c"
)

In [107]:
thread = client.beta.threads.create()

In [108]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 0 minutes 38 seconds
{
  "data": [
    {
      "id": "msg_ZsyskIJuFDnj9pLDnRSnmd3v",
      "assistant_id": "asst_ZYDRe1m4yshUFiuNMgo1C5Fd",
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "2023年のノーベル賞の最新の受賞者は、ノーベル経済学賞を受賞した「ゴールディン教授」とされています。詳細な氏名は情報から明確には伝わりませんが、ハーバード大学のゴールディン教授が男女の賃金格差の要因や労働市場における女性の役割の研究によって選ばれています。"
          },
          "type": "text"
        }
      ],
      "created_at": 1709541500,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_vYWmb9FbQLERDmUi0eEyvJ5k",
      "thread_id": "thread_utZtvYzVCv0hAyGi0MqGphJM"
    },
    {
      "id": "msg_6wRFCiBt11MQbY1iFd8ANcuc",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "以下の情報を元に、次の質問に答えてください。ノーベル賞の最新の受賞者は誰ですか？\n# 情報\n2023年のノーベル経済学賞は？\n男女の賃金格差の要因や労働市場における女性の役割などを研究したアメリカのハーバード大学のゴール

最新のノーベル賞受賞者は、ノーベル経済学賞を受賞したアメリカのハーバード大学のゴールディン教授です。これは最も最近の更新日である2023年10月9日の情報に基づいています。

###### Processed Prompt Tokens

![](comparison_prompt_tokens.PNG)

###### Processed Completion Tokens

![](comparison_completion_tokens.PNG)

###### Processed Inference Tokens

![](comparison_inference_tokens.PNG)

In [110]:
print("A:", 131 / 1000 * 4.513 + 224 / 1000 * 9.026, "円")
print("B:", 13.76 * 4.513 + 1.63 * 9.026, "円")
print("C:", 1.12 * 4.513 + 149 / 1000 * 9.026, "円")

A: 2.613027 円
B: 76.81126 円
C: 6.399434 円


### その他

#### 資料用に正規分布のグラフを作成する

##### 正規分布

In [5]:
request = "正規分布のグラフを表示してください。プロット以外の情報は全て消してください。"

In [6]:
assistant = client.beta.assistants.create(
    name="Demo",
    tools=[{"type": "code_interpreter"}],
    model="code-interpreter-model"
)

In [7]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_3qO4OFcBYjIWfyboc4utS5JT', created_at=1709008374, metadata={}, object='thread')


In [8]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 0 minutes 38 seconds
{
  "data": [
    {
      "id": "msg_ft3LJsAAslJqlpydeuUGF6lX",
      "assistant_id": "asst_1GCrrlv0xt9H32YmGsmLN3at",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-8otPoMqMDxdNEXfDGTQjOGXD"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the graph of a normal distribution with all plot information removed, as requested."
          },
          "type": "text"
        }
      ],
      "created_at": 1709008414,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_dSnXE3kIynByykOrJzrt1PUs",
      "thread_id": "thread_3qO4OFcBYjIWfyboc4utS5JT"
    },
    {
      "id": "msg_nsrXOxlTFhjfeAgAOtiaZC5Y",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "

In [9]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

print(messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_ft3LJsAAslJqlpydeuUGF6lX",
      "assistant_id": "asst_1GCrrlv0xt9H32YmGsmLN3at",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-8otPoMqMDxdNEXfDGTQjOGXD"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the graph of a normal distribution with all plot information removed, as requested."
          },
          "type": "text"
        }
      ],
      "created_at": 1709008414,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_dSnXE3kIynByykOrJzrt1PUs",
      "thread_id": "thread_3qO4OFcBYjIWfyboc4utS5JT"
    },
    {
      "id": "msg_nsrXOxlTFhjfeAgAOtiaZC5Y",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "正規分布のグラフを表示してください。プロット以外の情報は全て消してください。"
          },


In [10]:
data = json.loads(messages.model_dump_json(indent=2))
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)

assistant-8otPoMqMDxdNEXfDGTQjOGXD


In [11]:
content = client.files.content(image_file_id)

image = content.write_to_file("image.png")

![](gm.png)

##### 混合正規分布

In [12]:
request = "混合正規分布のグラフを表示してください。プロット以外の情報は全て消してください。"

In [13]:
get_response(client=client, assistant=assistant, thread=thread, request=request)

Status: completed
Elapsed time: 0 minutes 35 seconds
{
  "data": [
    {
      "id": "msg_1JyDHoRvvSges1YyaEA5auWT",
      "assistant_id": "asst_1GCrrlv0xt9H32YmGsmLN3at",
      "content": [
        {
          "image_file": {
            "file_id": "assistant-Fr8BdJw6Gt0rITPpw1iw9YeJ"
          },
          "type": "image_file"
        },
        {
          "text": {
            "annotations": [],
            "value": "Here is the graph of a mixed normal distribution with all plot information removed, as requested."
          },
          "type": "text"
        }
      ],
      "created_at": 1709008629,
      "file_ids": [],
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_qkS9UDW2fYHyW2IiRhcjd8Ek",
      "thread_id": "thread_3qO4OFcBYjIWfyboc4utS5JT"
    },
    {
      "id": "msg_EEnq1b6TIvpvXHhjTFR9aonc",
      "assistant_id": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "val

In [14]:
messages = client.beta.threads.messages.list(
    thread_id=thread.id
)

In [15]:
data = json.loads(messages.model_dump_json(indent=2))
image_file_id = data['data'][0]['content'][0]['image_file']['file_id']

print(image_file_id)

assistant-Fr8BdJw6Gt0rITPpw1iw9YeJ


In [16]:
content = client.files.content(image_file_id)

image = content.write_to_file("image.png")

![](gmm.png)